# Static Quantization

## 1. Import packages

In [1]:
import torch
import resnet
import copy
import torch.nn as nn
import helper
import torchvision
import torchvision.transforms as transforms
import os

In [2]:
cuda_device = torch.device("cuda:0")
cpu_device = torch.device("cpu:0")

## 2. Create a model instance

In [3]:
model_fp32 = resnet.resnet18(pretrained=False, num_classes=10)
model_fp32

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (relu2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     

In [4]:
state_dict = torch.load('resnet18.pt')
model_fp32.load_state_dict(state_dict)

<All keys matched successfully>

# CIFAR10 DataSet

In [5]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [6]:
num_workers = 8
    
train_set = torchvision.datasets.CIFAR10(root="data", train=True, download=False, transform=transform) 
test_set = torchvision.datasets.CIFAR10(root="data", train=False, download=False, transform=transform)

train_sampler = torch.utils.data.RandomSampler(train_set)
test_sampler = torch.utils.data.SequentialSampler(test_set)
    
train_loader = torch.utils.data.DataLoader(
    dataset=train_set, batch_size=32,
    sampler=train_sampler, num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(
    dataset=test_set, batch_size=32,
    sampler=test_sampler, num_workers=num_workers)

evaluate_model `int8_eval_loss` and `int8_eval_accurac`

In [7]:
fp32_eval_loss_tmp, fp32_eval_accuracy_tmp, len_test_dataset = helper.evaluate_model(model=model_fp32, test_loader=test_loader, device=cuda_device, criterion=None)
fp32_eval_accuracy_int = fp32_eval_accuracy_tmp.item()

print('fp32_eval_loss_tmp = {}'.format(fp32_eval_loss_tmp))
print('fp32_eval_accuracy_int = {}'.format(fp32_eval_accuracy_int))
print('len_test_dataset = {}'.format(len_test_dataset))

100%|██████████| 313/313 [00:04<00:00, 77.35it/s] 

fp32_eval_loss_tmp = 0
fp32_eval_accuracy_int = 9259
len_test_dataset = 10000


In [8]:
fp32_eval_loss = fp32_eval_loss_tmp/len_test_dataset
fp32_eval_accuracy = fp32_eval_accuracy_int/len_test_dataset

print(fp32_eval_loss)
print(fp32_eval_accuracy)

0.0
0.9259


## 3. Fusion layer

In [9]:
model_fp32

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (relu2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     

In [10]:
fused_model_fp32 = copy.deepcopy(model_fp32)
fused_model_fp32.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (relu2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     

In [11]:
# Fuse the activations to preceding layers, where applicable.
# This needs to be done manually depending on the model architecture.
# Common fusions include `conv + relu` and `conv + batchnorm + relu`
fused_model_fp32 = torch.quantization.fuse_modules(fused_model_fp32, [["conv1", "bn1", "relu"]], inplace=True)

for module_name, module in fused_model_fp32.named_children():
    if "layer" in module_name:
        for basic_block_name, basic_block in module.named_children():
            torch.quantization.fuse_modules(basic_block, [["conv1", "bn1", "relu"], ["conv2", "bn2"]], inplace=True)
            for sub_block_name, sub_block in basic_block.named_children():
                if sub_block_name == "downsample":
                    torch.quantization.fuse_modules(sub_block, [["0", "1"]], inplace=True)

fused_model_fp32

ResNet(
  (conv1): ConvReLU2d(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
  )
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (bn1): Identity()
      (relu): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): Identity()
      (skip_add): FloatFunctional(
        (activation_post_process): Identity()
      )
      (relu2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): ConvReLU2d(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
      (bn1): Identity()
      (relu): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3

## fused model equivalence

In [12]:
model_fp32.eval()
fused_model_fp32.eval()

assert helper.model_equivalence(model_1=model_fp32, model_2=fused_model_fp32, device=cpu_device, rtol=1e-05, atol=1e-05, num_tests=100, input_size=(1,3,32,32)), "Fused model is not equivalent to the original model!"

## 4. Static Quantization

In [13]:
class QuantizedResNet18(nn.Module):
    def __init__(self, model_fp32):
        super(QuantizedResNet18, self).__init__()
        
        self.quant = torch.quantization.QuantStub()
        self.model_fp32 = model_fp32
        self.dequant = torch.quantization.DeQuantStub()
        
    def forward(self, x):
        x = self.quant(x)
        x = self.model_fp32(x)
        x = self.dequant(x)
        return x

In [14]:
def calibrate_model(model, loader, device=torch.device("cpu:0")):

    model.to(device)
    model.eval()

    for inputs, labels in loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        _ = model(inputs)

In [15]:
# Quantization Model Define
quantized_model = QuantizedResNet18(model_fp32=fused_model_fp32)

# Quantization Configuration Define
quantized_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')     # 'qnnpack' for NVIDIA
torch.quantization.prepare(quantized_model, inplace=True)

# Calibration
calibrate_model(model=quantized_model, loader=test_loader, device=cuda_device)
quantized_model = quantized_model.to(cpu_device)

# Quantization Completed
quantized_model_int8 = torch.quantization.convert(quantized_model, inplace=True)
quantized_model_int8.eval()
print(quantized_model_int8)

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:177: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:897: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_begin // dst_bin_width, 0, self.dst_nbins - 1
/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:902: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' 

QuantizedResNet18(
  (quant): Quantize(scale=tensor([0.0407]), zero_point=tensor([59]), dtype=torch.quint8)
  (model_fp32): ResNet(
    (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.007564293686300516, zero_point=0, padding=(1, 1))
    (bn1): Identity()
    (relu): Identity()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.00538296764716506, zero_point=0, padding=(1, 1))
        (bn1): Identity()
        (relu): Identity()
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.008964813314378262, zero_point=60, padding=(1, 1))
        (bn2): Identity()
        (skip_add): QFunctional(
          scale=0.01268741674721241, zero_point=57
          (activation_post_process): Identity()
        )
        (relu2): ReLU(inplace=True)
      )
      (1): Bas

### MINI TEST

In [16]:
fp32_eval_loss_tmp, fp32_eval_accuracy_tmp, len_test_dataset = helper.evaluate_model(model=model_fp32, test_loader=test_loader, device=cuda_device, criterion=None)
fp32_eval_accuracy_int = fp32_eval_accuracy_tmp.item()
print('fp32_eval_loss_tmp = {}'.format(fp32_eval_loss_tmp))
print('fp32_eval_accuracy_int = {}'.format(fp32_eval_accuracy_int))
print('len_test_dataset = {}'.format(len_test_dataset))

fp32_eval_loss = fp32_eval_loss_tmp/len_test_dataset
fp32_eval_accuracy = fp32_eval_accuracy_int/len_test_dataset
print('\nfp32_eval_loss = {}'.format(fp32_eval_loss))
print('fp32_eval_accuracy = {}'.format(fp32_eval_accuracy))

100%|██████████| 313/313 [00:02<00:00, 138.41it/s]

fp32_eval_loss_tmp = 0
fp32_eval_accuracy_int = 9259
len_test_dataset = 10000

fp32_eval_loss = 0.0
fp32_eval_accuracy = 0.9259


In [17]:
int8_eval_loss_tmp, int8_eval_accuracy_tmp, len_test_dataset = helper.evaluate_model(model=quantized_model_int8, test_loader=test_loader, device=cpu_device, criterion=None)
int8_eval_accuracy_int = int8_eval_accuracy_tmp.item()
print('int8_eval_loss_tmp = {}'.format(int8_eval_loss_tmp))
print('int8_eval_accuracy_int = {}'.format(int8_eval_accuracy_int))
print('len_test_dataset = {}'.format(len_test_dataset))

int8_eval_loss = int8_eval_loss_tmp/len_test_dataset
int8_eval_accuracy = int8_eval_accuracy_int/len_test_dataset
print('\nint8_eval_loss = {}'.format(int8_eval_loss))
print('int8_eval_accuracy = {}'.format(int8_eval_accuracy))

100%|██████████| 313/313 [00:11<00:00, 26.75it/s]

int8_eval_loss_tmp = 0
int8_eval_accuracy_int = 9228
len_test_dataset = 10000

int8_eval_loss = 0.0
int8_eval_accuracy = 0.9228


In [31]:
quantized_model_int8

QuantizedResNet18(
  (quant): Quantize(scale=tensor([0.0407]), zero_point=tensor([59]), dtype=torch.quint8)
  (model_fp32): ResNet(
    (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.007564293686300516, zero_point=0, padding=(1, 1))
    (bn1): Identity()
    (relu): Identity()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): QuantizedConvReLU2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.00538296764716506, zero_point=0, padding=(1, 1))
        (bn1): Identity()
        (relu): Identity()
        (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.008964813314378262, zero_point=60, padding=(1, 1))
        (bn2): Identity()
        (skip_add): QFunctional(
          scale=0.01268741674721241, zero_point=57
          (activation_post_process): Identity()
        )
        (relu2): ReLU(inplace=True)
      )
      (1): Bas

# Test

In [18]:
# import urllib

# url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)

In [19]:
import time
from PIL import Image
from torchvision import transforms

input_image = Image.open("car.jpg")
preprocess = transforms.Compose([
    transforms.Resize(32),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# print(f"Input shape : {input_batch.shape}")
# print(f"Input : {input_batch[0][0][0][:14]}")

In [20]:
# output = model_fp32(input_batch)
output = quantized_model_int8(input_batch)

In [21]:
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

tensor([-0.5554,  5.6927, -0.8331, -0.6942, -0.9025, -0.7637, -0.7637, -0.8331,
        -0.4860,  0.0694])


In [22]:
categories = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

automobile 0.983534038066864
truck 0.0035532803740352392
ship 0.0020390532445162535
airplane 0.0019022977212443948
cat 0.0016556869959458709


In [23]:
model_fp32_cpu_inference_latency = helper.measure_inference_latency(model=model_fp32, device=cpu_device, input_size=(1,3,32,32), num_samples=100)
quantized_model_int8_cpu_inference_latency = helper.measure_inference_latency(model=quantized_model_int8, device=cpu_device, input_size=(1,3,32,32), num_samples=100)

print(f"model_fp32_cpu_inference_latency : {model_fp32_cpu_inference_latency}")
print(f"quantized_model_int8_cpu_inference_latency : {quantized_model_int8_cpu_inference_latency}")

model_fp32_cpu_inference_latency : 0.018832485675811767
quantized_model_int8_cpu_inference_latency : 0.009169454574584962


In [24]:
# torch.save(fused_model_fp32, 'fused_model_fp32.pth')
torch.save(quantized_model_int8, 'quantized_model_0728.pth')

In [25]:
def print_model_size(model):
    torch.save(model.state_dict(), "tmp.pt")
    print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

In [26]:
print_model_size(model_fp32)
print_model_size(quantized_model_int8)

44.77 MB
11.31 MB


In [28]:
torch.load('quantized_model_0728.pth')

AttributeError: 'ConvReLU2d' object has no attribute '_modules'